In [1]:
import os
import random
import re
from collections import Counter
from tqdm import tqdm

import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader

from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.metrics import f1_score

from transformers import (
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
    AutoTokenizer,
    AutoModelForSequenceClassification,
    AutoModelForCausalLM,
    pipeline
)

from datasets import load_dataset, Dataset
import evaluate

from konlpy.tag import Okt

from langchain.llms import OpenAI, HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.embeddings import HuggingFaceEmbeddings

from dataset import BERTDataset

ImportError: cannot import name 'BERTDataset' from 'dataset' (/opt/conda/lib/python3.10/site-packages/dataset/__init__.py)

In [ ]:
# ================================================
print("데이터셋 로드 및 설정")
# ================================================

# 랜덤 시드 설정
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)


# 모델과 토크나이저 로드
model_name = 'klue/bert-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)

df_train = pd.read_csv("/data/ephemeral/home/data/train.csv")
df_valid = pd.read_csv("/data/ephemeral/home/data/valid_output.csv")


# 데이터셋에 인덱스 컬럼 추가
df_train = df_train.reset_index().rename(columns={'index': 'idx'})
df_valid = df_valid.reset_index().rename(columns={'index': 'idx'})


데이터셋 로드 및 설정


In [ ]:
df_train_pc = df_train.copy()

In [2]:
#특수 기호 제거 함수:
def remove_special_characters(text):
    return re.sub(r'[^ ㄱ-ㅣ가-힣]', '', text)


#형태소 분석 및 불용어 제거 함수
okt = Okt()
korean_stopwords = set(['은', '는', '이', '가', '을', '를', '의', '에', '에서', '로', '으로', '과', '와', '도', '만', '에게', '께', '한테', '보다', '라고', '이라고', '으로서', '같이', '처럼', '만큼'])

def tokenize_and_remove_stopwords(text):
    tokens = okt.pos(text, stem=True)
    return ' '.join([word for word, pos in tokens if pos in ['Noun', 'Verb', 'Adjective'] and word not in korean_stopwords])

#2글자 이상 단어 필터링 함수
def filter_tokens_by_length(text, min_length=2):
    return ' '.join([word for word in text.split() if len(word) >= min_length])


#전체 전처리 함수
def preprocess_text(text):
    text = remove_special_characters(text)
    text = tokenize_and_remove_stopwords(text)
    text = filter_tokens_by_length(text)  # 2글자 이상 단어 필터링 추가
    return text


#DataFrame에 전처리 적용
def preprocess_dataframe(df):
    df['cleaned_text'] = df['text'].apply(preprocess_text)
    return df


# 데이터프레임 전처리
df_cleaned = preprocess_dataframe(df_train_pc)

# 정제된 문서 리스트 반환
cleaned_texts = df_cleaned['cleaned_text'].tolist()

JVMNotFoundException: No JVM shared library file (libjvm.so) found. Try setting up the JAVA_HOME environment variable properly.

In [5]:
pip install langchain-huggingface

Note: you may need to restart the kernel to use updated packages.


In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda


In [7]:
# 정제된 문서 리스트 반환
texts = df_cleaned['cleaned_text'].tolist()

In [8]:
#문자 임베딩
embeddings = HuggingFaceEmbeddings(model_name="nlpai-lab/KoE5")
text_embeddings = embeddings.embed_documents(texts)

/tmp/ipykernel_4077282/1460730533.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="nlpai-lab/KoE5")


In [9]:
kmeans = KMeans(n_clusters=7, random_state=42)


cluster_labels = kmeans.fit_predict(text_embeddings)

In [10]:
cluster_labels

array([2, 2, 5, ..., 1, 6, 5], dtype=int32)

In [11]:
def find_closest_text(cluster_center, embeddings, texts):
    distances = np.linalg.norm(embeddings - cluster_center, axis=1)
    closest_index = np.argmin(distances)
    return texts[closest_index]

cluster_centers = kmeans.cluster_centers_
representative_texts = [find_closest_text(center, text_embeddings, texts) for center in cluster_centers]

In [12]:
model_name = "beomi/llama-2-ko-7b"  # 또는 다른 한국어 Llama 모델
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

In [ ]:
hf_pipeline = pipeline(
    "text-generation",
    model=model, 
    tokenizer=tokenizer, 
    max_new_tokens=50,  # 출력 길이를 크게 줄임
    do_sample=True,
    temperature=0.5,  # 낮은 temperature로 더 결정적인 출력
    top_p=0.95,
    repetition_penalty=1.2
)
llm = HuggingFacePipeline(pipeline=hf_pipeline)

In [19]:
prompt_template = """뉴스 기사의 일부 단어입니다:

{text}

위 단어들과 가장 관련 있는 뉴스 도메인을 한 단어로만 답하세요. 예시: 정치, 경제, 사회, 문화, 국제, 과학, 스포츠.
다른 설명이나 추가 정보 없이 오직 도메인 단어 하나만 답하세요.

도메인:"""

prompt = PromptTemplate(template=prompt_template, input_variables=["text"])

In [20]:
domains = []
for text in representative_texts:
    input_text = prompt.format(text=text)
    print(f"Input text: {input_text}")  # 입력 텍스트 확인
    
    response = llm.invoke(input_text)
    print(f"Raw response: {response}")  # 원본 응답 확인
    
    # 응답 처리
    if isinstance(response, str):
        # 응답에서 '도메인:' 이후의 텍스트만 추출
        domain_parts = response.split('도메인:')
        if len(domain_parts) > 1:
            domain = domain_parts[-1].strip().split()[0]  # 첫 단어만 추출
        else:
            domain = "Unknown"
    else:
        domain = "Unknown"
    
    # 도메인 필터링
    if len(domain) > 15 or '://' in domain or domain.lower() in ['뉴스', '기사', '단어']:
        domain = "Unknown"
    
    print(f"Extracted domain: {domain}")  # 추출된 도메인 확인
    domains.append(domain)

print("Final domains:", domains)

Input text: 뉴스 기사의 일부 단어입니다:

언론 만남

위 단어들과 가장 관련 있는 뉴스 도메인을 한 단어로만 답하세요. 예시: 정치, 경제, 사회, 문화, 국제, 과학, 스포츠.
다른 설명이나 추가 정보 없이 오직 도메인 단어 하나만 답하세요.

도메인:
Raw response: 뉴스 기사의 일부 단어입니다:

언론 만남

위 단어들과 가장 관련 있는 뉴스 도메인을 한 단어로만 답하세요. 예시: 정치, 경제, 사회, 문화, 국제, 과학, 스포츠.
다른 설명이나 추가 정보 없이 오직 도메인 단어 하나만 답하세요.

도메인:​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​
Extracted domain: Unknown
Input text: 뉴스 기사의 일부 단어입니다:

출시 출리 오르다

위 단어들과 가장 관련 있는 뉴스 도메인을 한 단어로만 답하세요. 예시: 정치, 경제, 사회, 문화, 국제, 과학, 스포츠.
다른 설명이나 추가 정보 없이 오직 도메인 단어 하나만 답하세요.

도메인:
Raw response: 뉴스 기사의 일부 단어입니다:

출시 출리 오르다

위 단어들과 가장 관련 있는 뉴스 도메인을 한 단어로만 답하세요. 예시: 정치, 경제, 사회, 문화, 국제, 과학, 스포츠.
다른 설명이나 추가 정보 없이 오직 도메인 단어 하나만 답하세요.

도메인:​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​
Extracted domain: Unknown
Input text: 뉴스 기사의 일부 단어입니다:

서로

위 단어들과 가장 관련 있는 뉴스 도메인을 한 단어로만 답하세요. 예시: 정치, 경제, 사회, 문화, 국제, 과학, 스포츠.
다른 설명이나 추가 정보 없이 오직 도메인 단어 하나만 답하세요.

도메인:
Raw response: 뉴스 기사의 일부 단어입니다:

서로

위 단어들과 가장 관련 있는 뉴스 도메인을 한 단어로만 답하세요. 예시: 정치, 경제, 사회, 

In [21]:
print("Final domains:", domains)

Final domains: ['Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown']


In [ ]:
# beomi/llama-2-ko-7b"  
# ['다음은', '다음은', '다음은', '다음은', '다음은', '다음은', '다음은']

In [ ]:
#서버 필요없는 토큰 모델 제거

In [ ]:
# del tokenizer
# del model

In [ ]:
# import torch
# torch.cuda.empty_cache()

In [ ]:
# import gc
# gc.collect()

In [ ]:
# nlpai-lab/KoE5 모델은 트랜스포머 아키텍처를 기반으로 하며, 이는 자기주의(self-attention) 메커니즘을 사용하여 문장의 각 부분이 서로 어떻게 관련되어 있는지를 파악합니다
'''
미사 이용기 종보 토큰화 해서 
"미사"는 "이용기"와 "종보"와의 관계를 고려하여 표현됩니다.
"이용기"는 "미사"와 "종보"와의 관계를 고려합니다.
"종보"도 마찬가지로 다른 단어들과의 관계를 고려합니다.

의문1)

그렇다면 미사 이용기 종보는 뜻을 가지고 있지 않는 단어로 인식하지 않을 수 있음 그렇다면 단어가 손상 되었어도 임베딩 할때 중요도를 깨달을 수 있는 모델을 사용하는 것이 좋지 않을까?

서브워드 토큰화 (Subword Tokenization):
FastText와 같은 모델은 단어를 더 작은 단위(서브워드)로 분해하여 처리합니다3. 이 방식은 오타나 변형된 단어도 어느 정도 처리할 수 있게 해줍니다.
문자 수준 임베딩 (Character-level Embeddings):
문자 단위로 임베딩을 생성하는 방식입니다5. 이는 알 수 없는 단어나 오타에 대해서도 어느 정도의 의미를 추출할 수 있게 해줍니다.
바이트 페어 인코딩 (Byte Pair Encoding, BPE):
BPE는 자주 등장하는 문자 시퀀스를 하나의 토큰으로 취급합니다5. 이 방식은 알 수 없는 단어나 오타에 대해서도 유연하게 대응할 수 있습니다.
컨텍스트 기반 임베딩과 문자 수준 임베딩의 결합:
BERT나 ELMo와 같은 컨텍스트 기반 모델과 문자 수준 임베딩을 결합하는 방식을 고려할 수 있습니다13. 이렇게 하면 단어의 컨텍스트와 문자 수준의 정보를 모두 활용할 수 있습니다.
사전 처리 및 정규화:
텍스트 데이터에 대한 철저한 사전 처리와 정규화 과정을 거치면, 손상된 단어나 오타를 어느 정도 정정할 수 있습니다.
도메인 특화 임베딩:
특정 도메인에 특화된 임베딩 모델을 사용하거나 fine-tuning하는 것도 좋은 방법입니다3. 이는 해당 도메인에서 자주 등장하는 특수한 단어나 표현을 더 잘 처리할 수 있게 해줍니다.
이러한 접근 방식들은 단어가 손상되었거나 의미를 가지고 있지 않은 것처럼 보이는 경우에도 더 robust한 임베딩을 생성할 수 있게 해줍니다. 특히 서브워드 토큰화나 문자 수준 임베딩을 활용하면, 
"미사", "이용기", "종보"와 같은 단어들이 완전히 의미 없는 것으로 취급되지 않고, 그 구성 요소나 문자 수준에서의 패턴을 통해 어느 정도의 의미를 추출할 수 있습니다.


이러한 접근 방식들은 단어가 손상되었거나 의미를 가지고 있지 않은 것처럼 보이는 경우에도 더 robust한 임베딩을 생성할 수 있게 해줍니다. 특히 서브워드 토큰화나 문자 수준 임베딩을 활용하면, "미사", "이용기", "종보"와 같은 단어들이 완전히 의미 없는 것으로 취급되지 않고, 그 구성 요소나 문자 수준에서의 패턴을 통해 어느 정도의 의미를 추출할 수 있습니다.
결론적으로, 귀하의 제안처럼 단어가 손상되었거나 의미를 파악하기 어려운 경우에도 중요도를 인식할 수 있는 모델을 사용하는 것이 더 효과적일 수 있습니다. 이는 특히 실제 응용에서 노이즈가 있는 데이터나 도메인 특화된 용어를 다룰 때 매우 중요한 고려사항이 될 수 있습니다.


의문 2)
형태소로 구분을 할때 의미가 없는 단어들은 제거하는게 맞지 않을까?

주의할 점
도메인 특수성: 특정 도메인에서는 일반적인 불용어가 중요한 의미를 가질 수 있습니다.
문맥 손실: 과도한 단어 제거는 문장의 전체적인 문맥을 왜곡시킬 수 있습니다.
부정어 처리: '않다', '아니다' 등의 부정어는 문장의 의미를 완전히 바꿀 수 있으므로 주의가 필요합니다.
대부분의 NLP 작업에서 효과적일 것입니다. 다만, 특정 작업이나 도메인에 따라 이 전처리 과정을 약간 수정하거나 조정할 필요가 있을 수 있습니다.


 '날씨 새해 첫날 전국 가끔 구름 많다 아침 기온 영하',
 '개다 급전 패키지 나다 한정 판매',
 '애위 최고 스마트',
 '최대 트밤 의전',
 '고속 운행 차다',
 '스토어 유통 수수료 인하',
 '블블 출시',
 '대다 영정 담다 하다 종합',
 '사회 서비스 사무장 직원',
 '트럼프 주장 불법이민 전원 추방 미국 경제 축소',
 '불교 육식 금지 돼다',
 '최고 위원 출마 위해',
 '민주 평창올림픽 성공 개최 총력전 색깔론 한국 맹공',
 '창녕 육박 불볕 더위 물놀이 여름 축제 더위 탈출 종합',
 '해운 위기 용료 인하 합병 해결 되다',
 '대판 파라오 엘시 이집트 마지막 독립 언론 겁박',
 '이주열 통화정책 완화 정도 추가 조정 필요',
 '대다 드이 윈도 점율 최초',



'''

'\n미사 이용기 종보 토큰화 해서 \n"미사"는 "이용기"와 "종보"와의 관계를 고려하여 표현됩니다.\n"이용기"는 "미사"와 "종보"와의 관계를 고려합니다.\n"종보"도 마찬가지로 다른 단어들과의 관계를 고려합니다.\n\n의문1)\n\n그렇다면 미사 이용기 종보는 뜻을 가지고 있지 않는 단어로 인식하지 않을 수 있음 그렇다면 단어가 손상 되었어도 임베딩 할때 중요도를 깨달을 수 있는 모델을 사용하는 것이 좋지 않을까?\n\n서브워드 토큰화 (Subword Tokenization):\nFastText와 같은 모델은 단어를 더 작은 단위(서브워드)로 분해하여 처리합니다3. 이 방식은 오타나 변형된 단어도 어느 정도 처리할 수 있게 해줍니다.\n문자 수준 임베딩 (Character-level Embeddings):\n문자 단위로 임베딩을 생성하는 방식입니다5. 이는 알 수 없는 단어나 오타에 대해서도 어느 정도의 의미를 추출할 수 있게 해줍니다.\n바이트 페어 인코딩 (Byte Pair Encoding, BPE):\nBPE는 자주 등장하는 문자 시퀀스를 하나의 토큰으로 취급합니다5. 이 방식은 알 수 없는 단어나 오타에 대해서도 유연하게 대응할 수 있습니다.\n컨텍스트 기반 임베딩과 문자 수준 임베딩의 결합:\nBERT나 ELMo와 같은 컨텍스트 기반 모델과 문자 수준 임베딩을 결합하는 방식을 고려할 수 있습니다13. 이렇게 하면 단어의 컨텍스트와 문자 수준의 정보를 모두 활용할 수 있습니다.\n사전 처리 및 정규화:\n텍스트 데이터에 대한 철저한 사전 처리와 정규화 과정을 거치면, 손상된 단어나 오타를 어느 정도 정정할 수 있습니다.\n도메인 특화 임베딩:\n특정 도메인에 특화된 임베딩 모델을 사용하거나 fine-tuning하는 것도 좋은 방법입니다3. 이는 해당 도메인에서 자주 등장하는 특수한 단어나 표현을 더 잘 처리할 수 있게 해줍니다.\n이러한 접근 방식들은 단어가 손상되었거나 의미를 가지고 있지 않은 것처럼 보이는 경우에도 더 robust한 임베딩을 